In [1]:
import netgen.gui

import netgen.geom2d as geom2d
from ngsolve import *

%gui tk

In [2]:
geometry=geom2d.SplineGeometry()
points=[(0,0),(1,0),(1,1),(0,1),
        (0.3,0.5),(0.5,0.5),(0.5,0.7),(0.3,0.7)]
geo_points=[geometry.AppendPoint(*point) for point in points]

lines= [(0,1,1,2,0),(1,2,1,2,0),
       (2,3,1,2,0),(3,0,1,2,0),
       
       (4,5,2,1,2),(5,6,3,1,2),
       (6,7,4,1,2),(7,4,5,1,2)]

for p1,p2,bc,left,right in lines:
    geometry.Append(['line',geo_points[p1],geo_points[p2]], bc=bc,
                   leftdomain=left,rightdomain=right)

In [3]:
mesh = Mesh(geometry.GenerateMesh (maxh=0.05))

In [4]:
Draw(mesh)

In [5]:
order = 2

fes = H1(mesh, order=order, dirichlet=[1])

a = BilinearForm(fes)
f = LinearForm(fes)

u = fes.TrialFunction()
v = fes.TestFunction()

lamda = DomainConstantCF([1,10])
fval = DomainConstantCF([1,0])

a += SymbolicBFI ( lamda * u.Deriv() * v.Deriv() )
f += SymbolicLFI ( fval * v )

a.Assemble()
f.Assemble()

u = GridFunction(fes)

u.vec.data = a.mat.Inverse(fes.FreeDofs()) * f.vec

In [6]:
dudx = GridFunction(fes)
dudy = GridFunction(fes)
dudx.Set(lamda*u.Deriv()[0])
dudy.Set(lamda*u.Deriv()[1])

bndcoefx = DomainConstantCF([0,0, 1, 0, -1])
bndcoefy = DomainConstantCF([0,-1, 0, 1, 0])

In [7]:
int = Integrate(dudx*bndcoefx+dudy*bndcoefy, mesh, definedon=mesh.Boundaries('.*'))
print("1", -int)

Draw(u, mesh, "gfu")
Draw(dudx, mesh, "dudxinterp")
Draw(dudy, mesh, "dudyinterp")
Draw(u.Deriv()[0], mesh, "dudx")
Draw(u.Deriv()[1], mesh, "dudy")
Draw(lamda*u.Deriv()[0], mesh, "dudxl")
Draw(lamda*u.Deriv()[1], mesh, "dudyl")

1 0.039882568065711776
